In [1]:
# These lines of code import important libraries and the 'create_engine' function from 'sqlalchemy.' First, 'pandas' is imported and given the alias 'pd' for data manipulation. Then, the entire 'sqlalchemy' library, used for relational database tasks, is imported. The 'os' library comes next, simplifying file and directory operations. 're' is imported from Python's standard library, allowing manipulation of strings using patterns. Lastly, 'create_engine' is imported from 'sqlalchemy' to establish a database engine for connecting to relational databases.
import pandas as pd, sqlalchemy, os, re
from sqlalchemy import create_engine

# This code is defining a Python dictionary named db_config that stores the configuration details for connecting to a database. The purpose of this code is to define a configuration dictionary that can be used later in the code to establish a connection to the specified database. 
db_config = {
    "user": "root",
    "password": "WHATSOEVER30",
    "host": "localhost",
    "port": 3306,  # Default MySQL port
    "database": "survey"
}

# This code creates an SQLAlchemy database engine using the provided configuration information. The create_engine function combines all these elements into a connection string and uses it to create an SQLAlchemy engine. This engine can be used to execute SQL queries and interact with the specified MySQL database using the connection details.
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# This code is defining the location of the folder where the Excel files to be processed by the program are stored.
folder_path = 'survey/'

# This code defines a list called keywords that contains specific words or terms of interest. The purpose of this list is to identify and match these keywords within the code later on. This list allows for flexibility by considering variations in capitalization, meaning that the code will match these keywords regardless of whether they are in uppercase, lowercase, or a combination of both.
keywords = ["inpatient", "warded", "ip", "inp", "in"]

# This code creates a list named month_order that contains abbreviated month names in a specific order. This list is designed to hold the abbreviated month names in a specific sequence, likely to match the ordering of months used in data files or databases where the code is processing or organising data.
month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# The code begins by iterating through files in the directory specified by folder_path.
for filename in os.listdir(folder_path):

# It checks if the current file being iterated has the '.xlsx' file extension using filename.endswith('.xlsx').
    if filename.endswith('.xlsx'):

# If the file has a valid Excel extension, it attempts to extract the month name from the filename using a regular expression (re.match). This is done to identify the relevant month for the data in the file.
        month_match = re.match(r'([A-Za-z]+)\.xlsx', filename)

# If a month name is successfully extracted from the filename and it matches one of the months in the month_order list, the code proceeds to the next step.
        if month_match:
            month_name = month_match.group(1)
            if month_name in month_order:

# The code reads the Excel file into a dictionary of data frames using Pandas (pd.read_excel). Each sheet in the Excel file becomes a data frame within the dictionary.
                excel_data = pd.read_excel(os.path.join(folder_path, filename), sheet_name=None)
                
# It then iterates through each sheet and data frame in the dictionary.
                for sheet_name, data_frame in excel_data.items():
                    
# For each sheet, it checks if any of the keywords in the keywords list can be found in the sheet name. The check is case-insensitive.
                    if any(keyword in sheet_name.lower() for keyword in keywords):
                        
# If a keyword is found in the sheet name, it constructs a table name based on the month's position in the month_order list. It uses this table name to store the data from the current sheet.
                        table_name = f"in{month_order.index(month_name) + 1:02d}"
                        
# The data from the sheet is written to a MySQL database table using the to_sql method, and any existing table with the same name is replaced.
                        data_frame.to_sql(table_name, con=engine, if_exists="replace", index=False)
                        
# A message is printed to indicate that the data from the current file has been successfully uploaded to the corresponding table.
                        print(f"Data from '{filename}' has been successfully uploaded to the '{table_name}' table.")

# After processing all Excel files, a final message is printed to indicate that all data frames have been uploaded to the MySQL database as tables in order.
print("All data frames have been uploaded to the MySQL database as tables in order.")


Data from 'Apr.xlsx' has been successfully uploaded to the 'in04' table.
Data from 'Aug.xlsx' has been successfully uploaded to the 'in08' table.
Data from 'Dec.xlsx' has been successfully uploaded to the 'in12' table.
Data from 'Feb.xlsx' has been successfully uploaded to the 'in02' table.
Data from 'Jan.xlsx' has been successfully uploaded to the 'in01' table.
Data from 'Jul.xlsx' has been successfully uploaded to the 'in07' table.
Data from 'Jun.xlsx' has been successfully uploaded to the 'in06' table.
Data from 'Mar.xlsx' has been successfully uploaded to the 'in03' table.
Data from 'May.xlsx' has been successfully uploaded to the 'in05' table.
Data from 'Nov.xlsx' has been successfully uploaded to the 'in11' table.
Data from 'Oct.xlsx' has been successfully uploaded to the 'in10' table.
Data from 'Sep.xlsx' has been successfully uploaded to the 'in09' table.
All data frames have been uploaded to the MySQL database as tables in order.


In [ ]:
from sqlalchemy import create_engine

# Define your database connection details
db_config = {
    "user": "root",
    "password": "WHATSOEVER30",
    "host": "localhost",
    "port": 3306,
    "database": "anl503-eca"
}

# Create an SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Define the list of tables to be combined
table_names = ["in01", "in02", "in03", "in04", "in05", "in06", "in07", "in08", "in09", "in10", "in11", "in12"]

# Create the SQL query to combine the tables using UNION ALL
combine_tables_query = "CREATE TABLE doc_survey AS\n"

# Iterate through the table names and generate UNION ALL statements
for table_name in table_names:
    combine_tables_query += f"SELECT * FROM {table_name}\nUNION ALL\n"

# Remove the last "UNION ALL" statement
combine_tables_query = combine_tables_query[:-11]  # Remove the last 11 characters (length of "UNION ALL\n")

# Execute the SQL query to create the doc_survey table
with engine.connect() as connection:
    connection.execute(combine_tables_query)

print("The doc_survey table has been created by combining data from in01 to in12.")

# Define a dictionary to map the original column names to the new names and descriptions
column_info = {
    "X1": ("respectful","How_courteous_and_respectful_the_doctors_were (1=Very Poor; 2=Poor; 3=Satisfactory; 4=Good; 5=Excellent; 99=NA)"),
    "X2": ("seems_competent", "How well the doctors displayed professional knowledge and skills (1=Very Poor; 2=Poor; 3=Satisfactory; 4=Good; 5=Excellent; 99=NA)"),
    "X3": ("emphathises_well", "How well the doctors showed care and concern (1=Very Poor; 2=Poor; 3=Satisfactory; 4=Good; 5=Excellent; 99=NA)"),
    "X4": ("listens_well", "How well the doctors understood your concerns (1=Very Poor; 2=Poor; 3=Satisfactory; 4=Good; 5=Excellent; 99=NA)"),
    "X5": ("explains_and_updates_well", "How well the doctors provided clear explanation and updates on care and treatment (1=Very Poor; 2=Poor; 3=Satisfactory; 4=Good; 5=Excellent; 99=NA)"),
    "Y": ("overall_sat", "How satisfied you were with the medical treatment you received (1=Very Poor; 2=Poor; 3=Satisfactory; 4=Good; 5=Excellent; 99=NA)"),
    "AGE": ("age_bracket", "Age bracket (1/2/3=[17,29]; 4=[30,39]; 5=[40,49]; 6=[50,59]; 7=[60,64]; 8=[65,inf]; 99=RF)"),
    "GENDER": ("sex", "Gender (1=M; 2=F)"),
    "WTYPE": ("ward_type", "Ward type (1=A1; 2=B1; 3=B2; 4=C)")
}

# Generate SQL statements to rename columns and add descriptions in the doc_survey table
alter_table_query = f"ALTER TABLE doc_survey"

for old_name, (new_name, description) in column_info.items():
    alter_table_query += f" CHANGE {old_name} {new_name} VARCHAR(255) COMMENT '{description}',"

# Remove the trailing comma
alter_table_query = alter_table_query.rstrip(',')

# Execute the SQL query to rename columns and add descriptions in the doc_survey table
with engine.connect() as connection:
    connection.execute(alter_table_query)

print("The column names in the doc_survey table have been updated with descriptions.")

# SQL statement to update '99' values to NULL
update_query = """
UPDATE doc_survey
SET respectful = NULLIF(respectful, '99'),
    seems_competent = NULLIF(seems_competent, '99'),
    emphathises_well = NULLIF(emphathises_well, '99'),
    listens_well = NULLIF(listens_well, '99'),
    explains_and_updates_well = NULLIF(explains_and_updates_well, '99'),
    overall_sat = NULLIF(overall_sat, '99'),
    age_bracket = NULLIF(age_bracket, '99'),
    sex = NULLIF(sex, '99'),
    ward_type = NULLIF(ward_type, '99');
"""

# Execute the update query to replace '99' with NULL
with engine.connect() as connection:
    connection.execute(update_query)

print("Replaced '99' values with NULL in the doc_survey table.")
